In [1]:
!pip install warcio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 1.5 MB/s eta 0:00:00


In [2]:
import requests
import json
import re
from urllib.parse import quote_plus
from warcio.archiveiterator import ArchiveIterator
from bs4 import BeautifulSoup

In [19]:
SERVER = 'http://index.commoncrawl.org/'
INDEX_NAME = 'CC-MAIN-2024-33'

target_site = 'https://www.gorodperm.ru/'  # Сайт для поиска.

# Ключевые слова для поиска.
keywords = [
    r'г\. Пермь',
    'Пермский Политех',
    'кафедра ИТАС',
    'МГУ им. Ломоносова',
    'МФТИ им. Баумана',
]

# Пары ключевых слов для поиска в одном предложении.
keyword_pairs = [
    ('Борис Пастернак', 'Пермь'),
]


In [4]:
# Функция поиска по индексу Common Crawl.
def search_cc_index(url):
    encoded_url = quote_plus(f'*.{url}/*')
    index_url = f'{SERVER}{INDEX_NAME}-index?url={encoded_url}&output=json'
    response = requests.get(index_url)
    if response.status_code == 200:
        records = response.text.strip().split('\n')
        return [json.loads(record) for record in records]
    else:
        return None

In [5]:
def fetch_page_from_cc(record):
    offset, length = int(record['offset']), int(record['length'])
    s3_url = f'https://data.commoncrawl.org/{record["filename"]}'

    byte_range = f'bytes={offset}-{offset+length-1}'
    response = requests.get(s3_url, headers={'Range': byte_range}, stream=True)

    if response.status_code == 206:
        stream = ArchiveIterator(response.raw)
        for warc_record in stream:
            if warc_record.rec_type == 'response':
                return warc_record.content_stream().read()
    else:
        print(f"Не удалось загрузить данные: {response.status_code}")
        return None

    return None

In [10]:
# Функция поиска ключевых слов в контенте.
def find_keywords_in_content(content, keywords, keyword_pairs):
    found_matches = []
    soup = BeautifulSoup(content, 'html.parser')
    text = soup.get_text(separator=' ')

    # print(f"\nОбработанный текст (первые 500 символов): {text[:500]}...\n")

    # Ищем отдельные ключевые слова.
    for keyword in keywords:
        if re.search(keyword, text, re.IGNORECASE):
            found_matches.append((text.strip(), keyword))

    # Ищем пары ключевых слов в одном предложении.
    sentences = re.split(r'[.!?]', text)
    for sentence in sentences:
        for kw1, kw2 in keyword_pairs:
            if kw1.lower() in sentence.lower() and kw2.lower() in sentence.lower():
                found_matches.append((sentence.strip(), f'{kw1} + {kw2}'))

    return found_matches

In [20]:
records = search_cc_index(target_site)
if not records:
    print(f"Записей не найдено для сайта: {target_site}")

if records:
    print(f"Найдено {len(records)} записей для сайта {target_site}")
    for idx, record in enumerate(records):
        print(f"Обработка записи {idx + 1} из {len(records)}: {record['url']}")
        content = fetch_page_from_cc(record)
        if content:
            found_phrases = find_keywords_in_content(content, keywords, keyword_pairs)
            if found_phrases:
                print(f"\nСтраница: {record['url']}")
                for phrase, keyword in found_phrases:
                    print(f"Ключевое слово: {keyword}")
                    print(f"Контекст: {phrase[:30]}\n")
                print("="*80)
        else:
            print("Не удалось получить контент страницы.\n")

Найдено 3212 записей для сайта https://www.gorodperm.ru/
Обработка записи 1 из 3212: https://www.gorodperm.ru/

Страница: https://www.gorodperm.ru/
Ключевое слово: г\. Пермь
Контекст: Муниципальное образование горо

Обработка записи 2 из 3212: https://gorodperm.ru/

Страница: https://gorodperm.ru/
Ключевое слово: г\. Пермь
Контекст: Муниципальное образование горо

Обработка записи 3 из 3212: https://gorodperm.ru/about/brends/

Страница: https://gorodperm.ru/about/brends/
Ключевое слово: г\. Пермь
Контекст: Узнаваемая Пермь - Муниципальн

Обработка записи 4 из 3212: https://www.gorodperm.ru/about/Brothers_/Associations_of_municipalities/

Страница: https://www.gorodperm.ru/about/Brothers_/Associations_of_municipalities/
Ключевое слово: г\. Пермь
Контекст: Объединения муниципальных обра

Обработка записи 5 из 3212: https://www.gorodperm.ru/about/history/

Страница: https://www.gorodperm.ru/about/history/
Ключевое слово: г\. Пермь
Контекст: История возникновения Перми - 

Обработка записи

KeyboardInterrupt: 